In [69]:
import mediapipe as mp
import cv2
import os
import numpy as np

In [70]:
# Path save
SAVE_PATH = r"C:/Users/ronde/PROJECTS/ASL_TO_TEXT/data/images"
IMG_SIZE = 227  # Make sure this matches your model's input size

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

def normalize_landmarks(landmarks):
    x_coords = [lm[0] for lm in landmarks]
    y_coords = [lm[1] for lm in landmarks]
    
    min_x, max_x = min(x_coords), max(x_coords)
    min_y, max_y = min(y_coords), max(y_coords)
    
    normalized_landmarks = [
        [(x - min_x) / (max_x - min_x), (y - min_y) / (max_y - min_y), z]
        for x, y, z in landmarks
    ]
    return normalized_landmarks

def preprocess_landmarks(landmarks, img_size=IMG_SIZE):
    normalized_landmarks = normalize_landmarks(landmarks)
    
    landmarks_image = np.zeros((img_size, img_size, 3), dtype=np.uint8)
    for lm in normalized_landmarks:
        x, y = int(lm[0] * (img_size - 1)), int(lm[1] * (img_size - 1))
        cv2.circle(landmarks_image, (x, y), 10, (255, 0, 0), 3)
    
    return landmarks_image

letter_or_word = input("Enter the letter or word to train: ").strip()
output_folder = os.path.join(SAVE_PATH, letter_or_word)
os.makedirs(output_folder, exist_ok=True)

cam = cv2.VideoCapture(0)
if not cam.isOpened():
    print("Error: Camera not accessible.")
    exit()

frame_count = 0
capturing = False

while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame.")
        break
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = [(landmark.x, landmark.y, landmark.z) for landmark in hand_landmarks.landmark]
            preprocessed_image = preprocess_landmarks(landmarks)
            
            cv2.imshow('Preprocessed Hand', preprocessed_image)
            
            if capturing:
                # Capture 500 images in a burst
                for _ in range(500): 
                    img_name = os.path.join(output_folder, f"{letter_or_word}_{frame_count}.jpg")
                    cv2.imwrite(img_name, preprocessed_image)
                    frame_count += 1
                print(f"500 Images saved to: {output_folder}")
                capturing = False # Stop capturing after the burst

    cv2.imshow('Hand Tracking', frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):
        capturing = True
        print("Started capturing...")

    elif key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

Started capturing...
500 Images saved to: C:/Users/ronde/PROJECTS/ASL_TO_TEXT/data/images\Please
